## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-31-09-39-53 +0000,bbc,"Putin, Modi and other world leaders in China f...",https://www.bbc.com/news/videos/c5y310qngqpo?a...
1,2025-08-31-09-30-00 +0000,wsj,Yale’s Trendsetting Private-Equity Strategy Is...,https://www.wsj.com/finance/investing/yales-tr...
2,2025-08-31-09-30-00 +0000,wsj,Taylor Swift and Tariffs Can Both Help Sell Di...,https://www.wsj.com/finance/stocks/taylor-swif...
3,2025-08-31-09-30-00 +0000,wsj,Warren Buffett’s Son Battles Colombia’s Boomin...,https://www.wsj.com/world/americas/howard-buff...
4,2025-08-31-09-30-00 +0000,wsj,"Households earning between $50,000 and $100,00...",https://www.wsj.com/economy/consumers/the-midd...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2320/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
87,trump,26
5,china,9
579,israeli,9
215,killed,9
161,gaza,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
100,2025-08-30-18-46-00 +0000,wsj,The Trump administration is drawing up plans t...,https://www.wsj.com/politics/national-security...,56
29,2025-08-31-07-40-59 +0000,nypost,Trump slams Pritzker on Chicago crime after at...,https://nypost.com/2025/08/31/us-news/trump-sl...,50
95,2025-08-30-19-37-30 +0000,bbc,Trump administration moves to fire most Voice ...,https://www.bbc.com/news/articles/cdd38v3zvjdo...,49
34,2025-08-31-05-02-48 +0000,bbc,"Reeling from Trump's tariffs, India and China ...",https://www.bbc.com/news/articles/c0mlen3grx7o...,48
63,2025-08-31-00-05-56 +0000,nypost,Chicago mayor orders police not to cooperate w...,https://nypost.com/2025/08/30/us-news/chicago-...,48


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
100,56,2025-08-30-18-46-00 +0000,wsj,The Trump administration is drawing up plans t...,https://www.wsj.com/politics/national-security...
81,40,2025-08-30-21-58-05 +0000,nypost,Tens of thousands of Palestinians flee amid Ga...,https://nypost.com/2025/08/30/world-news/tens-...
98,38,2025-08-30-19-00-34 +0000,nypost,Graphic video shows Israeli father throw himse...,https://nypost.com/2025/08/30/world-news/video...
21,32,2025-08-31-09-00-00 +0000,wsj,Democratic Party moderates see Virginia govern...,https://www.wsj.com/politics/elections/in-virg...
78,28,2025-08-30-22-23-30 +0000,nypost,"NYC shooting vics were ‘targeted,’ police sour...",https://nypost.com/2025/08/30/us-news/nyc-shoo...
16,26,2025-08-31-09-01-31 +0000,nyt,Chevron’s Boss Says the World Will Need Oil fo...,https://www.nytimes.com/2025/08/31/business/ch...
109,26,2025-08-30-17-25-31 +0000,nypost,Russia launches sweeping attack on Southern Uk...,https://nypost.com/2025/08/30/world-news/russi...
87,24,2025-08-30-20-39-24 +0000,bbc,Houthis confirm their prime minister killed in...,https://www.bbc.com/news/articles/c620ykrxedwo...
6,22,2025-08-31-09-21-41 +0000,cbc,China's Xi and India's Modi vow to resolve bor...,https://www.cbc.ca/news/world/china-india-regi...
95,21,2025-08-30-19-37-30 +0000,bbc,Trump administration moves to fire most Voice ...,https://www.bbc.com/news/articles/cdd38v3zvjdo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
